In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
df = pd.concat([train,test],ignore_index=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 366
Data columns (total 13 columns):
ApplicantIncome      981 non-null int64
CoapplicantIncome    981 non-null float64
Credit_History       902 non-null float64
Dependents           956 non-null object
Education            981 non-null object
Gender               957 non-null object
LoanAmount           954 non-null float64
Loan_Amount_Term     961 non-null float64
Loan_ID              981 non-null object
Loan_Status          614 non-null object
Married              978 non-null object
Property_Area        981 non-null object
Self_Employed        926 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 107.3+ KB


In [5]:
df['Gender'].fillna('other',inplace=True)
df['Genderc'] = df['Gender'].astype('category')
df['Genderc'] = df['Genderc'].cat.rename_categories([1,2,3])
df=df.drop("Gender", axis=1)

In [6]:
df["Married"].fillna(value="No", inplace=True)
df["Marriedc"]=df["Married"].astype("category")
df["Marriedc"]=df["Marriedc"].cat.rename_categories([1,2])
df=df.drop("Married", axis=1)

In [7]:
df["Dependents"].fillna(value="0", inplace=True)
df["Dependentsc"]=df["Dependents"].astype("category")
df["Dependentsc"]=df["Dependentsc"].cat.rename_categories([1,2,3,4])
df=df.drop("Dependents", axis=1)

In [8]:
df["Educationc"]=df["Education"].astype("category")
df["Educationc"]=df["Educationc"].cat.rename_categories([1,2])
df=df.drop("Education", axis=1)

In [9]:
df["Self_Employed"].fillna(value="other", inplace=True)
df["Self_Employedc"]=df["Self_Employed"].astype("category")
df["Self_Employedc"]=df["Self_Employedc"].cat.rename_categories([1,2,3])
df=df.drop("Self_Employed", axis=1)

In [10]:
df["Property_Areac"]=df["Property_Area"].astype("category")
df["Property_Areac"]=df["Property_Areac"].cat.rename_categories([1,2,3])
df=df.drop("Property_Area", axis=1)

In [11]:
df["Credit_History"].fillna(value=1, inplace=True)
df["Credit_Historyc"]=df["Credit_History"].astype("category")
df=df.drop("Credit_History", axis=1)

In [12]:
df["ApplicantIncome"]=df["ApplicantIncome"].astype("object")
df["CoapplicantIncome"]=df["CoapplicantIncome"].astype("object")

In [13]:
df["LoanAmount"]=df["LoanAmount"].fillna(df["LoanAmount"].mean())
df["LoanAmount"]=df["LoanAmount"].astype("object")

In [14]:
df["Loan_Amount_Term"]=df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].mean())
df["Loan_Amount_Term"]=df["Loan_Amount_Term"].astype("object")

In [15]:
from sklearn.preprocessing import scale
df['EMI']=df["LoanAmount"]/df["Loan_Amount_Term"]

In [16]:
df["sum_Income"]=df["ApplicantIncome"] + (df["CoapplicantIncome"])
df["Poly_feature"]=df['EMI']/df["sum_Income"]
df["ratio"]=df["sum_Income"]/df["LoanAmount"]

In [17]:
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Loan_ID,Loan_Status,Genderc,Marriedc,Dependentsc,Educationc,Self_Employedc,Property_Areac,Credit_Historyc,EMI,sum_Income,Poly_feature,ratio
0,5849,0,142.512,360,LP001002,Y,2,1,1,1,1,3,1.0,0.395865,5849,6.76809e-05,41.0423
1,4583,1508,128,360,LP001003,N,2,2,2,1,1,1,1.0,0.355556,6091,5.83739e-05,47.5859
2,3000,0,66,360,LP001005,Y,2,2,1,1,2,3,1.0,0.183333,3000,6.11111e-05,45.4545
3,2583,2358,120,360,LP001006,Y,2,2,1,2,1,3,1.0,0.333333,4941,6.74627e-05,41.175
4,6000,0,141,360,LP001008,Y,2,1,1,1,1,3,1.0,0.391667,6000,6.52778e-05,42.5532


In [18]:
df_train=df[df.Loan_Status.notnull()]
df_test=df[df.Loan_Status.isnull()]

In [19]:
df_test=df_test.drop("Loan_Status", axis=1)
key=df_test["Loan_ID"]
df_test=df_test.drop("Loan_ID", axis=1)
df_train=df_train.drop("Loan_ID", axis=1)

In [20]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
Xtrsp=pd.DataFrame(enc.fit_transform(df_train[["Genderc","Marriedc","Dependentsc","Educationc","Self_Employedc","Property_Areac", "Credit_Historyc"]]).toarray())
Xtstsp=pd.DataFrame(enc.transform(df_test[["Genderc","Marriedc","Dependentsc","Educationc","Self_Employedc","Property_Areac", "Credit_Historyc"]]).toarray())
print(Xtrsp.shape)

(614, 19)


In [21]:
df_train["Loan_Statusc"]=df_train["Loan_Status"].astype("category")
df_train["Loan_Statusc"]=df_train["Loan_Statusc"].cat.rename_categories([1,0])

In [22]:
y_train=df_train["Loan_Statusc"]
df_train=df_train.drop("Loan_Status", axis=1)
df_train=df_train.drop("Loan_Statusc", axis=1)

In [23]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
sel = SelectKBest(chi2, k=10)
Xtrsp=pd.DataFrame(sel.fit_transform(Xtrsp,y_train))
Xtstsp=pd.DataFrame(sel.transform(Xtstsp))

In [24]:
Xtrf=df_train[["Poly_feature","sum_Income","LoanAmount","Loan_Amount_Term","ratio"]]
Xtstf=df_test[["Poly_feature","sum_Income","LoanAmount","Loan_Amount_Term","ratio"]]

In [25]:
Xtrf.index=Xtrsp.index
X_train=pd.concat([Xtrsp,Xtrf], axis=1)
Xtstf.index=Xtstsp.index
X_test=pd.concat([Xtstsp,Xtstf], axis=1)

In [27]:
from sklearn.linear_model import LogisticRegression 
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import RandomForestClassifier 
#from sklearn.naive_bayes import GaussianNB
#from sklearn.neural_network import MLPClassifier
#from sklearn.svm import LinearSVC

#model=RandomForestClassifier(n_estimators=128, verbose=1)
model=LogisticRegression(C=0.01, tol=0.00001)
#model=KNeighborsClassifier(n_neighbors=40)
#model=LinearSVC(C=0.01, tol=0.0000001)
#model=GaussianNB()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(y_pred)

out=pd.DataFrame()
out["Loan_ID"]=key
out["Loan_Status"]=y_pred
out["Loan_Status"]=out["Loan_Status"].astype("category")
out["Loan_Status"]=out["Loan_Status"].cat.rename_categories(["Y","N"])
out.to_csv('outcredit.csv', index=False)

[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
